# Step 17: V15 — Transit-window asymmetry

Goal: check whether the flux baseline around the transit is asymmetric (a ramp/step proxy).

Why this matters:
- Strong asymmetry can indicate instrumental ramps, pointing drifts, or other systematics that can bias transit depth/shape.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V15')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "asymmetry_sigma": 1.678,
    "baseline": 0.999969,
    "mu_left": -1.7e-05,
    "mu_right": -8.6e-05,
    "n_bins_half": 12,
    "n_left_bins": 12,
    "n_left_points": 1942,
    "n_right_bins": 12,
    "n_right_points": 1943,
    "window_mult": 2.0
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V15: Asymmetry
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_asymmetry
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='17_v15_asymmetry')
    run_path = run_out_dir / 'V15_asymmetry.png'
    docs_path = (docs_out_dir / 'V15_asymmetry.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_asymmetry(r, ax=ax)
    ax.set_title('V15: Transit-window asymmetry')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/17_v15_asymmetry/V15_asymmetry.png`

![V15: Asymmetry](../artifacts/tutorial_toi-5807-incremental/17_v15_asymmetry/V15_asymmetry.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/17_v15_asymmetry/V15_asymmetry.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/17_v15_asymmetry/V15_asymmetry.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none.
- **Result:** asymmetry ≈1.68σ (mu_left ≈ -1.7e-05, mu_right ≈ -8.6e-05).
- **Interpretation:** no strong evidence for a ramp/step systematic in the transit window.
- **Next step:** extended diagnostics (V16–V19).

</details>
